In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-airline-sentiment/Tweets.csv
/kaggle/input/twitter-airline-sentiment/database.sqlite
/kaggle/input/us-baby-names/StateNames.csv
/kaggle/input/us-baby-names/NationalReadMe.pdf
/kaggle/input/us-baby-names/hashes.txt
/kaggle/input/us-baby-names/NationalNames.csv
/kaggle/input/us-baby-names/StateReadMe.pdf
/kaggle/input/us-baby-names/database.sqlite


In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.express as px
import os
from termcolor import colored
path = "../input/us-baby-names/"  #Insert path here
database = path + 'database.sqlite'
print(database)
conn = sql.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
print(colored("Display the most common names ending with 'rlie' and having a minimum count of more than 1000 in descending order", 'blue',attrs=['bold','dark','underline']))

pd.read_sql("""SELECT name,sum(count) as counts 
                FROM StateNames 
                where name like '%rlie' 
                group by name 
                having counts >1000 
                order by counts desc""", conn)



../input/us-baby-names/database.sqlite
Display the most common names ending with 'rlie' and having a minimum count of more than 1000 in descending order


,Name,counts
0,Charlie,150729
1,Pearlie,14217
2,Carlie,11162
3,Karlie,7578
4,Arlie,3277
5,Harlie,1796
6,Marlie,1428
7,Earlie,1405
8,Kimberlie,1375


In [3]:
print(colored("Display the count of the name Marlie with DOB as 1999 by doing an Inner join on nationalnames and statenames tables and applying appropriate filtering", 'blue',attrs=['bold','dark','underline']))

pd.read_sql("""select n.id as NationalnamesID, n.year as NationalnamesYear, n.gender as NationalnamesGender, n.count as NationalnamesCount,  s.id as StatenamesID, s.year as StatenamesYear, s.gender as StatenamesGender, s.count as StatenamesCount  
                from nationalnames as n 
                inner join statenames as s  
                using(year) 
                where n.name='Marlie' and s.Name='Marlie' and year=1999""", conn)





Display the count of the name Marlie with DOB as 1999 by doing an Inner join on nationalnames and statenames tables and applying appropriate filtering


,NationalnamesID,NationalnamesYear,NationalnamesGender,NationalnamesCount,StatenamesID,StatenamesYear,StatenamesGender,StatenamesCount
0,1306583,1999,F,55,3711681,1999,F,5
1,1306583,1999,F,55,4885308,1999,F,5


In [4]:
print(colored("Adding an extra analysis column which makes use of the case statements of whether count is > 1000 or > 2000 or > 5000 ", 'blue',attrs=['bold','dark','underline']))

pd.read_sql("""select name,count, 
case when count>5000 then 'More than 5000'
     when count>2000 then 'More than 2000 and less than 5000' 
     when count>1000 then 'More than 1000 and less than 2000' 
     else count 
     end as analysis 
     from nationalnames limit 5""", conn)

Adding an extra analysis column which makes use of the case statements of whether count is > 1000 or > 2000 or > 5000 


,Name,Count,analysis
0,Mary,7065,More than 5000
1,Anna,2604,More than 2000 and less than 5000
2,Emma,2003,More than 2000 and less than 5000
3,Elizabeth,1939,More than 1000 and less than 2000
4,Minnie,1746,More than 1000 and less than 2000


In [5]:
print(colored("By using Common Table Expressions(CTEs) & case statements, getting the baby names starting with Mar% and DOB 1910, comparing the count column of statenames vs nationalnames table and adding a CountAnalysis column indicating which table count is higher", 'blue',attrs=['bold','dark','underline']))

pd.read_sql("""with cte as (select name,sum(count) as CountofStatenames from statenames where name like 'Mar%' and year=1910 group by name) 
                select name,cte.CountofStatenames,nationalnames.count as CountofNationalnames, 
                case 
                    when cte.CountofStatenames > nationalnames.count then "Count in Statenames table is greater" 
                    else "Count in Nationalnames table is greater" 
                    end as CountAnalysis 
                    from nationalnames 
                    join cte 
                    using(name) 
                    where nationalnames.name in (select cte.name from cte) and year=1910 group by name""", conn)


By using Common Table Expressions(CTEs) & case statements, getting the baby names starting with Mar% and DOB 1910, comparing the count column of statenames vs nationalnames table and adding a CountAnalysis column indicating which table count is higher


,Name,CountofStatenames,CountofNationalnames,CountAnalysis
0,Marceline,6,12,Count in Nationalnames table is greater
1,Marcella,257,295,Count in Nationalnames table is greater
2,Marcia,17,57,Count in Nationalnames table is greater
3,Marcus,20,63,Count in Nationalnames table is greater
4,Margaret,8229,8226,Count in Statenames table is greater
5,Margaretta,12,21,Count in Nationalnames table is greater
6,Margarita,46,60,Count in Nationalnames table is greater
7,Margarito,10,14,Count in Nationalnames table is greater
8,Margery,14,66,Count in Nationalnames table is greater
9,Margie,276,305,Count in Nationalnames table is greater


In [6]:
print(colored("By using subquery, getting the distinct name of the babies in Nationalnames by subsetting the babies born in FL from statenames tables", 'blue',attrs=['bold','dark','underline']))

pd.read_sql("""select distinct name 
                from Nationalnames 
                where name in (select distinct name from statenames where state='FL')   """, conn)


By using subquery, getting the distinct name of the babies in Nationalnames by subsetting the babies born in FL from statenames tables


,Name
0,Aaden
1,Aadhya
2,Aadya
3,Aadyn
4,Aaiden
...,...
10600,Zylah
10601,Zyler
10602,Zymarion
10603,Zyon


In [7]:
print(colored("SUBQUERY SECTION \n", 'blue',attrs=['bold','dark','underline']))
print(colored("By using subquery, Top 5 names of the babies whose count is 200 more than the average count", 'blue',attrs=['bold','dark','underline']))
    
pd.read_sql("""select * from nationalnames 
                where count > (select avg(count)+200 from nationalnames) 
                limit 5""", conn)
    

SUBQUERY SECTION 

By using subquery, Top 5 names of the babies whose count is 200 more than the average count


,Id,Name,Year,Gender,Count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746


In [8]:
print(colored("SUBQUERY SECTION \n", 'blue',attrs=['bold','dark','underline']))
print(colored("For how many years the name Mary and Minnie has appeared in statenames table. - By using a subquery in select caluse - Method 1", 'blue',attrs=['bold','dark','underline']))

pd.read_sql("""select distinct name, 
                (select count(distinct year) from statenames where statenames.name = nationalnames.name and name in ('Mary','Minnie')) as count 
                from nationalnames 
                where name in ('Mary','Minnie') """, conn)




SUBQUERY SECTION 

For how many years the name Mary and Minnie has appeared in statenames table. - By using a subquery in select caluse - Method 1


,Name,count
0,Mary,105
1,Minnie,94


In [9]:
print(colored("SUBQUERY SECTION \n", 'blue',attrs=['bold','dark','underline']))
print(colored("For how many years the name Mary and Minnie has appeared in statenames table. - By joining tables - Method 2", 'blue',attrs=['bold','dark','underline']))
pd.read_sql("""select distinct nationalnames.name, count(distinct statenames.year) from nationalnames 
                join statenames 
                on statenames.name = nationalnames.name 
                where nationalnames.name in ('Mary','Minnie') 
                group by nationalnames.name """, conn)


SUBQUERY SECTION 

For how many years the name Mary and Minnie has appeared in statenames table. - By joining tables - Method 2


,Name,count(distinct statenames.year)
0,Mary,105
1,Minnie,94


In [10]:

print(colored("SUBQUERY SECTION \n", 'blue',attrs=['bold','dark','underline']))
print(colored("For how many years the name Mary and Minnie has appeared in statenames table. - By using a subquery in from clause - Method 3", 'blue',attrs=['bold','dark','underline']))
pd.read_sql("""select distinct nationalnames.name, subq.cntyear 
                from nationalnames, 
                (select name,count(distinct year) as cntyear from statenames group by name ) as subq where subq.name=nationalnames.name and nationalnames.name in ('Mary','Minnie') """, conn)

SUBQUERY SECTION 

For how many years the name Mary and Minnie has appeared in statenames table. - By using a subquery in from clause - Method 3


,Name,cntyear
0,Mary,105
1,Minnie,94
